In [ ]:
#@title TRABAJO FINAL


!pip install graphviz
from graphviz import Digraph
import ipywidgets as widgets
from IPython.display import display
from graphviz import Digraph
import graphviz
import copy
import math

def hash_list(c1, c2):
  return "cols: " + str(c1) + " | vals: " + str(c2) 

def get_val_fact(map, cols, vals):
  return map[hash_list(cols, vals)]

def readFile(file):
  finput = open(file,"r")
  lines = finput.readlines()
  n_i = len(lines) - 1
  names = lines[0].strip().split(",")
  df_names = []
  for i in range(1, len(lines)):
    row = lines[i].strip().split(",")
    df_names.append(row)
  
  return n_i, names, df_names

class dataF:

  def __init__(self,n_i,names,df_names,ed=[]):

    self.n_i = n_i
    self.names = names
    self.n = len(self.names)
    self.uniq = [set() for x in range(self.n)]
    self.df_names = df_names
    
    # uniq values
    for i in range(self.n_i):
      row = df_names[i]
      for j in range(self.n):
        self.uniq[j].add(row[j])

    self.hiper_param = 1
    
    #mapear elementos
    self.mapElems()

    # get ad and in list
    self.lad = [[] for x in range(self.n)] 
    self.lin = [[] for x in range(self.n)]

    for elem in ed:
      u = self.mnames[elem[0]]
      v = self.mnames[elem[1]]
      self.lad[u].append(v)
      self.lin[v].append(u)

    self.initParents()
    self.getFactors()

    self.edges = []
    for i in range(self.n):
      u = self.names[i]
      for x in self.lad[i]:
        v = self.names[x]
        self.edges.append([u, v])
  
  def mapElems(self):
    # map columns
    self.card = []
    self.mnames = dict()
    for x in range(self.n):
      self.mnames[self.names[x]] = x
    
    self.mcvals = [dict() for x in range(self.n)]
    for c in range(self.n):
      self.uniq[c] = list(self.uniq[c])
      self.card.append(len(self.uniq[c]))
      for i in range(self.card[c]):
        self.mcvals[c][self.uniq[c][i]] = i
    
    self.df = []
    for row in self.df_names:
      aux = []
      for c in range(self.n):
        aux.append(self.mcvals[c][row[c]])
      
      self.df.append(aux)

  def initParents(self):  
    self.parent = [-1]*self.n
    flag = [0]*self.n

    def dfs(u, p, vis):
      if(vis[u]):
        return 
      
      vis[u] = True
      self.parent[u] = p
      for x in self.lad[u]:
        dfs(x, u, vis) 

    self.root = -1
    for x in range(self.n):
      for y in self.lad[x]:
        flag[y] = 1
    
    for x in range(self.n):
      if(not flag[x]):
        self.root = x
        break

    vis = [False]*self.n
    dfs(self.root, -1, vis)

  def getFactors(self):
    self.mapFact = dict()
    for i in range(self.n):
      cols = [i] + self.lin[i] 
      fact = self.calcFactor(i)
      for x in fact:
        self.mapFact[hash_list(cols, x[:-1])] = x[-1]

  def reset(self):
    self.initParents()
    self.getFactors()

  def rep(self, params):
    cnt = 0
    for row in self.df:
      ok = True
      for x in params:
        if(row[x[0]] != x[1]):
            ok = False
      if(ok):
        cnt+= 1
      
    return cnt

  def prob_general(self, params, factor):
    # (m[value] + p) / (M + (p * card(column))
    exp1 = self.rep(params) + self.hiper_param
    exp2 = self.n_i + (self.hiper_param * factor)
    return round(exp1/exp2,6) 
      
  def calcFactor(self, id):

    def normalize(arr):
      s = 0
      ans = []

      for x in arr:
        s+= x
      
      for x in arr:
        ans.append(x/s)
      
      return ans
      
    def add_column(M, col):
      for i in range(len(col)):
        M[i].append(col[i])

    cols = [id] + self.lin[id]

    uniqs = []
    cnt = []
    ids = []
    vuelta = []
    total = 1

    for x in cols:

        uniqs.append([i for i in range(self.card[x])])
        ids.append(total)
            
        total*= self.card[x]
        
        cnt.append(0)
        vuelta.append(0)

    D = []
    c_ans = []
    c_norm = []
    M = []

    for u in range(total):
        vals = []
        i = 0
        for x in cols:
            vals.append([x, uniqs[i][cnt[i]]])
            i+= 1
        
        M.append( [e[1] for e in vals] )

        exp1 = self.prob_general(vals, total)
        exp2 = self.prob_general(vals[1:], total/self.card[id])

        if(len(self.lin[id]) == 0):
          D.append(exp1)
        
        else:
          D.append(exp1/exp2)

        for j in range(len(cols)):
            vuelta[j]+= 1
            if(vuelta[j] == ids[j]):
                cnt[j] = (cnt[j] + 1)%len(uniqs[j])
                vuelta[j] = 0

        if(len(D) == self.card[id]):
          c_ans+= D
          c_norm+= normalize(D)
          D = []
          
    # Agregando columnas
    add_column(M, c_norm)

    return M


def Entropia(D):
  ans = 0
  for i in range(D.n):
    combs = D.calcFactor(i)
    for x in range(len(combs)):
      combs[x] = combs[x][:-1]

    for comb in combs:
      params = [[i, comb[0]]]
      for id in range(1, len(comb)):
        params.append([ D.lin[i][id - 1] , comb[id]])
      

      Nijk = D.rep(params)
      a = Nijk/D.n_i
      
      params = params[1:]
      den = D.n_i
      
      if(params):
        den = D.rep(params)

      if(Nijk == 0):
        continue

      b = math.log2(Nijk/den)

      ans+= a*b
  
  return -D.n_i*ans

def AIC(D):
  K = 0
  for i in range(D.n):
    q = 1
    for x in D.lin[i]:
      q *= D.card[x]

    K+= (D.card[i]-1)*q
  
  return Entropia(db) + K

def MDL(D):
  K = 0
  for i in range(D.n):
    q = 1
    for x in D.lin[i]:
      q *= D.card[x]

    K+= (D.card[i]-1)*q
  
  return Entropia(db) + K*math.log2(D.n_i)/2.0

def hk2(D):

  def fact(num):
    ans = 1
    while(num > 0):
      ans*= num
      num-= 1
    
    return ans

  ans = 1
  for i in range(D.n):
    P = D.parent[i]
    if(P != -1):
      for j in range(D.card[P]):
        Nij = D.rep([ [ P , j ] ] )
        term = 1
        for k in range(D.card[i]):
          Nijk = D.rep([ [P, j] , [i , k ] ])
          term*= fact(Nijk)

        
        term*= fact(D.card[i] - 1)/fact(D.card[i] - 1 + Nij)
        
        ans*= term

  return ans

def K2(D, lb, ub, metrica, order=None):

  def getEdges(nodes_parents, i=None, parents_i={}):
    ed = []

    for x in nodes_parents:
      for p in nodes_parents[x]:
        ed.append([p,x])
    
    if(i != None):
      for p in parents_i:
        ed.append([p, i])

    return ed


  nodes = D.names if order is None else order
  
  nodes_parents = {x: [] for x in nodes}
  scores = {'AIC': AIC, 'Entropy': Entropia, 'MDL': MDL, 'K2': hk2}


  for i in range(len(nodes)):
      parents_i = []
      
      P_old = scores[metrica](dataF(D.n_i, D.names, D.df_names, getEdges(nodes_parents, nodes[i], parents_i)))
      OKToProceed = True

      while OKToProceed and len(parents_i) < ub and i > 0:
        pred = [ nodes[ii] for ii in range(i)]

        maxs = []

        for node in pred:
          maxs.append(scores[metrica](dataF(D.n_i, D.names, D.df_names, getEdges(nodes_parents, nodes[i], parents_i + [node]))))
    
        mx = max(maxs)
        z = pred[maxs.index(mx)]
        P_new = mx

        if P_new > P_old:
          P_old = P_new
          parents_i.append(z)

        else:
          if(len(parents_i) < lb):
            parents_i.append(z)

          OKToProceed = False

      nodes_parents[nodes[i]] = parents_i
      # print("Nodo: ", nodes[i], " padres de Nodo => ", parents_i)

  return dataF(D.n_i, D.names, D.df_names, getEdges(nodes_parents))

def greedyLS(D, metrica):

  def cycle(u, G, vis, reStack):
    if(not vis[u]):
      vis[u] = True
      reStack[u] = True

      for x in G.lad[u]:
        if(not vis[x] and cycle(x, G, vis, reStack)):
          return True
        elif(reStack[x]):
          return True
      
    reStack[u] = False
    return False

  def isValid(G):
    vis = [False]*G.n
    reStack = [False]*G.n 

    for i in range(G.n):
      if(cycle(i, G, vis, reStack)):
        return False
    
    return True
    
  # Extrayendo lista de nodos
  nodes = D.names
    
  # Scores
  scores = {'AIC': AIC, 'Entropy': Entropia, 'MDL': MDL, 'K2': hk2}
 
  # 1. σbest ← σi
  omega_best = copy.deepcopy(D)
  score_best = scores[metrica](omega_best)

  # 2. do
  while True:
      
    # 3. σ ← σbest
    O = copy.deepcopy(omega_best)

    # 4.
    progress = False

    # 5. for each operator o ∈ O
    for node in nodes:
          
      cases = []
      
      for edge in O.edges:
        aux = [e for e in O.edges if e != edge]
        cases.append(dataF(O.n_i, O.names, O.df_names, aux))
        cases.append(dataF(O.n_i, O.names, O.df_names, aux + [[edge[1], edge[0]]]))
          
      for node2 in nodes:
        if([node, node2] not in O.edges and node != node2):
          cases.append(dataF(O.n_i, O.names, O.df_names, O.edges + [[node, node2]] ))         

      for case in cases:
        if isValid(case):
          score_o = scores[metrica](case)
          if score_o > score_best:
            omega_best = copy.deepcopy(case)
            score_best = score_o
            progress = True
 
    if not progress:
      break
    
  return omega_best

class InterMensajes:

  def __init__(self, D, A , a):
    
    self.D = D
    self.A = A
    self.a = a

    self.mapProb = dict()
    self.lamb = [[0]*self.D.card[x] for x in range(self.D.n)]
    self.lambP = [[0]*self.D.card[self.D.parent[x]] if self.D.parent[x] != -1 else [] for x in range(self.D.n) ] 

    self.pi = [[0]*self.D.card[x] for x in range(self.D.n)]
    self.piP = [[0]*self.D.card[self.D.parent[x]] if self.D.parent[x] != -1 else [] for x in range(self.D.n) ]

  def enviarMensajePi(self, z, x):
    
    # subfunciones
    def multHijos(z, zval, excl):
      ans = 1
      for y in self.D.lad[z]:
        if(y not in excl):
          ans*= self.lambP[y][zval]
      
      return ans

    # alg
    for elem in range(self.D.card[z]):
      self.piP[x][elem] = self.pi[z][elem]*multHijos(z, elem, [x])
      # print(self.piP[x][elem])
    
    auxProb = []
    for elem in range(self.D.card[x]):
      val = 0
      for elemP in range(self.D.card[z]):
        val+= get_val_fact(self.D.mapFact, [x, z], [elem, elemP])*self.piP[x][elemP]
      
      self.pi[x][elem] = val
      # print(self.pi[x][elem])
      auxProb.append(self.lamb[x][elem]*self.pi[x][elem])
    
    #normalizado
    for elem in range(self.D.card[x]):
      # print(auxProb[elem])
      self.mapProb[hash_list([x] + self.A, [elem] + self.a)] = auxProb[elem]/sum(auxProb)
    
    # for elem in range(self.D.card[x]):
    #   print("normal: " , get_val_fact(self.mapProb, [x] + self.A, [elem] + self.a))
    
    # print("end", )

    for y in self.D.lad[x]:
      if(y not in self.A):
        self.enviarMensajePi(x, y)
  
  def InicializarArbol(self):
    for x in range(self.D.n):
      for elem in range(self.D.card[x]):
        self.lamb[x][elem] = 1

      if(self.D.parent[x] != -1):
        for elem in range(self.D.card[self.D.parent[x]]):
          self.lambP[x][elem] = 1
      

    for x in range(self.D.card[self.D.root]):
      val = get_val_fact(self.D.mapFact,[self.D.root],[x])

      self.mapProb[hash_list([self.D.root] + self.A, [x] + self.a)] = val
      self.pi[self.D.root][x] = val

    for x in self.D.lad[self.D.root]:
      self.enviarMensajePi(self.D.root, x)

  def ActualizarArbol(self, V, v):
    self.A.append(V)
    self.a.append(v)

    self.lamb[V][v] = 1
    self.pi[V][v] = 1
    self.mapProb[hash_list([V] + self.A, [v] + self.a)] = 1

    for elem in range(self.D.card[V]):
      if(elem != v):
        self.lamb[V][elem] = 0
        self.pi[V][elem] = 0
        self.mapProb[hash_list([V] + self.A, [elem] + self.a)] = 0
    
    if(V != self.D.root and self.D.parent[V] not in self.A):
      self.enviarMensajeLamb(V, self.D.parent[V]) #hijo padre
    
    for x in self.D.lad[V]:
      if(x not in self.A):
        self.enviarMensajePi(V, x) 
  
  def enviarMensajeLamb(self, y, x):
    
    auxProb = []

    for elem in range(self.D.card[x]):
      val = 0
      for elemH in range(self.D.card[y]):
        val+= get_val_fact(self.D.mapFact, [y, x], [elemH, elem])*self.lamb[y][elemH]

      self.lambP[y][elem] = val
      # print(val)

      val = 1
      for H in self.D.lad[x]:
        val*= self.lambP[H][elem]
      
      self.lamb[x][elem] = val 
      # print(val)

      auxProb.append(self.lamb[x][elem]*self.pi[x][elem])
      # print(auxProb[-1])
    
    # normalizar
    for elem in range(self.D.card[x]):
      self.mapProb[hash_list([x] + self.A, [elem] + self.a)] = auxProb[elem]/sum(auxProb)
      # print("normal: ", auxProb[elem]/sum(auxProb))
    

    # print("stop")

    if(x != self.D.root and self.D.parent[x] not in self.A):
      self.enviarMensajeLamb(x, self.D.parent[x])
    
    for w in self.D.lad[x]:
      if(w != y and w not in self.A):
        self.enviarMensajePi(x, w)

  def query(self, var):
    nvar = self.D.names[var]
    
    # print("Lambda de Padre de " + nvar + ": " , self.lambP[var])
    # print("Pi de Padre de " + nvar + ": ", self.piP[var])
    # print("Lambda de " + nvar + ": " , self.lamb[var])
    # print("Pi de " + nvar + ": ", self.pi[var])

    aux = []
    for elem in range(self.D.card[var]):  
      aux.append(get_val_fact(self.mapProb, [var] + self.A, [elem] + self.a))
    
    # print("Probabilidad dado A :", aux)

    return nvar, self.lambP[var], self.piP[var], self.lamb[var], self.pi[var], aux

# Interfaz

tab_nest = widgets.Tab()

upl_data = widgets.FileUpload(accept='.csv',multiple=False,description="Select data.csv")
loaded = False
uploaded = False

btn_upl_data = widgets.Button(
    description='Upload DATASET',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
    )

hiper_param_slider = widgets.FloatSlider(
    value=1,
    min=0,
    max=10,
    #step=1,
    description='α',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

btn_create = widgets.Button(
    description='CREATE COMPONENTS',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check',
    layout=widgets.Layout(width="17%")
    )

# INTERFAZ HIPERPARAMETRO
hiperparam = widgets.FloatSlider(description="α")

def upl_data_file(a):
  global uploaded
  global upl_data
  if upl_data.value == {}:
    print("No hay ningun archivo subido.")
  else:
    uploaded_filename = next(iter(upl_data.value))
    content = upl_data.value[uploaded_filename]['content']
    with open('data.csv', 'wb') as f: f.write(content)
    uploaded = True
    print("Archivo subido")
    upl_data.value.clear()
    upl_data._counter = 0

def on_change(v):
    global hiper_param
    global db
    hiper_param_slider.max = db.n_i
    hiper_param = v['new']
    db.hiper_param = hiper_param

def load_components(a):
    global uploaded
    global db
    global hiperparam
    global tab_nest

    if uploaded == False:
        print("Suba un archivo para crear los componentes")
        return

    # SHOW HIPER PARAM
    upl_box = widgets.VBox([upl_data_box,btn_create,hiper_param_slider])
    tab_nest.children = [upl_box]
    tab_nest.set_title(0, 'UPLOAD')
    tab_nest.set_title(1, 'DATASET')

    # SHOW DATAFRAME
    n_i, names, df_names = readFile('data.csv')
    db = dataF(n_i,names,df_names)
    db.hiper_param = float(hiperparam.value)
    rows_boxes = list()
    labels=list()
    for head in db.names:
      label = widgets.Button(description = head, disabled = True, button_style='info')
      labels.append(label)
    headers_box = widgets.HBox(labels)

    for row in db.df_names:
      values=list()
      for val in row:
        value = widgets.Button(description = val, disabled = True)
        values.append(value)
      rows_boxes.append(widgets.HBox(values))
    dataset_box = widgets.VBox([headers_box,*rows_boxes])

    # SHOW K2
    childs_k2 = list()
    params_k2 = list()
    graph_k2 = list()

    metrica_k2_drop = widgets.Dropdown(options=['Entropy', 'AIC', 'MDL', 'K2'],
                              value="Entropy",
                              description="Metrica",)
    params_k2.append(metrica_k2_drop)
    
    for i in range(db.n):
      drop = widgets.Dropdown(options=db.names,
                              value=db.names[i],
                              description="{}:".format(i+1),)
      params_k2.append(drop)

    lb_k2_drop = widgets.FloatSlider(value=1,
                                  min=0,
                                  max=db.n,
                                  step=1,
                                  description='Min Padres:',
                                  disabled=False,
                                  continuous_update=False,
                                  orientation='horizontal',
                                  readout=True,
                                  readout_format='d')
    
    mb_k2_drop = widgets.FloatSlider(value=1,
                                  min=1,
                                  max=db.n,
                                  step=1,
                                  description='Max Padres:',
                                  disabled=False,
                                  continuous_update=False,
                                  orientation='horizontal',
                                  readout=True,
                                  readout_format='d')

    params_k2.append(lb_k2_drop)
    params_k2.append(mb_k2_drop)

    k2_button = widgets.Button(
      description='Calcular',
      disabled=False,
      button_style='info',
      tooltip='',
      icon='check')
    
    
    def k2_button_click(b):
      graph_k2 = []
      index_max = len(params_k2)-3
      order = []
      for i in range(1,index_max-1):
        order.append(params_k2[i].value)
        for j in range(i+1,index_max):
          if params_k2[i].value == params_k2[j].value:
            print("Todos los valores tienen que ser diferentes")
            return
      order.append(params_k2[index_max-1].value)
      dK2 = K2(db, lb_k2_drop.value, mb_k2_drop.value, metrica_k2_drop.value, order)
      dot = Digraph(format='png')
      for head in dK2.names:
        dot.node(head)
      for edge in dK2.edges:
        parent = edge[0] 
        child = edge[1]
        dot.edge(parent, child)
      dot.render("k2")
      file_image = open("k2.png", "rb")
      image = file_image.read()
      image_widget = widgets.Image(value=image, format='png', layout=widgets.Layout(width='400px'))
      graph_k2.append(image_widget)
      right_k2_vbox = widgets.VBox(graph_k2)
      tab_k2 = widgets.HBox([left_k2_vbox, right_k2_vbox])
      list_tabs = list(tab_nest.children)
      list_tabs.pop(2)
      list_tabs.insert(2, tab_k2)
      tab_nest.children = tuple(list_tabs)
    
    k2_button.on_click(k2_button_click)
    params_k2.append(k2_button)
    childs_k2.append(widgets.VBox(params_k2))
    left_k2_vbox = widgets.HBox(childs_k2)
    right_k2_vbox = widgets.VBox(graph_k2)
    tab_k2 = widgets.HBox([left_k2_vbox, right_k2_vbox])

    # SHOW GREEDY
    greedy_db = dataF(db.n_i, db.names, db.df_names)
    childs_greedy = list()
    params_greedy = list()
    graph_greedy = list()
    red_greedy = list()

    metrica_greedy_drop = widgets.Dropdown(options=['Entropy', 'AIC', 'MDL', 'K2'],
                              value="Entropy",
                              description="Metrica",)
    params_greedy.append(metrica_greedy_drop)

    parent_greedy_param = widgets.Dropdown(options=greedy_db.names,
                              value=greedy_db.names[0],
                              description="Padre:")
    
    child_greedy_param = widgets.Dropdown(options=greedy_db.names,
                              value=greedy_db.names[0],
                              description="Hijo:")
    
    edges_greedy_box = widgets.HBox([parent_greedy_param, child_greedy_param])
    params_greedy.append(edges_greedy_box)

    add_greedy_button = widgets.Button(
      description='Agregar',
      disabled=False,
      button_style='info',
      tooltip='',
      icon='check')
    
    delete_greedy_button = widgets.Button(
      description='Eliminar',
      disabled=False,
      button_style='info',
      tooltip='',
      icon='check')

    greedy_button = widgets.Button(
      description='Calcular',
      disabled=False,
      button_style='info',
      tooltip='',
      icon='check')
    
    red_greedy = []
    dot = Digraph(format='png')
    for head in greedy_db.names:
      dot.node(head)
    for edge in greedy_db.edges:
      parent = edge[0] 
      child = edge[1]
      dot.edge(parent, child)
    dot.render("red_greedy")
    file_image = open("red_greedy.png", "rb")
    image = file_image.read()
    image_widget = widgets.Image(value=image, format='png', layout=widgets.Layout(width='400px'))
    red_greedy.append(image_widget)
    
    def add_greedy_edge(b):
      parent = parent_greedy_param.value
      child = child_greedy_param.value

      if parent == child:
        print("El padre y el hijo debe ser diferente")
        return

      for edge in greedy_db.edges:
        if edge[0] == parent and edge[1] == child:
          print("Ya existe la conexion")
          return
      
      greedy_db.edges.append([parent, child])

      red_greedy = []
      dot = Digraph(format='png')
      for head in greedy_db.names:
        dot.node(head)
      for edge in greedy_db.edges:
        parent = edge[0] 
        child = edge[1]
        dot.edge(parent, child)
      dot.render("red_greedy")
      file_image = open("red_greedy.png", "rb")
      image = file_image.read()
      image_widget = widgets.Image(value=image, format='png', layout=widgets.Layout(width='400px'))
      red_greedy.append(image_widget)
      params_greedy.pop()
      params_greedy.append(widgets.HBox(red_greedy))
      childs_greedy = []
      childs_greedy.append(widgets.VBox(params_greedy))
      left_greedy_vbox = widgets.HBox(childs_greedy)
      tab_greedy = widgets.HBox([left_greedy_vbox, right_greedy_vbox])
      list_tabs = list(tab_nest.children)
      list_tabs.pop(3)
      list_tabs.insert(3, tab_greedy)
      tab_nest.children = tuple(list_tabs)

      print("Agregado: " + parent + " -> " + child)

    def delete_greedy_edge(b):
      parent = parent_greedy_param.value
      child = child_greedy_param.value

      if parent == child:
        print("El padre y el hijo debe ser diferente")
        return

      for edge in greedy_db.edges:
        if edge[0] == parent and edge[1] == child:
          greedy_db.edges.remove(edge)

          red_greedy = []
          dot = Digraph(format='png')
          for head in greedy_db.names:
            dot.node(head)
          for edge in greedy_db.edges:
            parent = edge[0] 
            child = edge[1]
            dot.edge(parent, child)
          dot.render("red_greedy")
          file_image = open("red_greedy.png", "rb")
          image = file_image.read()
          image_widget = widgets.Image(value=image, format='png', layout=widgets.Layout(width='400px'))
          red_greedy.append(image_widget)
          params_greedy.pop()
          params_greedy.append(widgets.HBox(red_greedy))
          childs_greedy = []
          childs_greedy.append(widgets.VBox(params_greedy))
          left_greedy_vbox = widgets.HBox(childs_greedy)
          tab_greedy = widgets.HBox([left_greedy_vbox, right_greedy_vbox])
          list_tabs = list(tab_nest.children)
          list_tabs.pop(3)
          list_tabs.insert(3, tab_greedy)
          tab_nest.children = tuple(list_tabs)

          print("Eliminado: " + parent + " -> " + child)
          return
        
      print("No existe la conexion")
    
    def greedy_button_click(b):
      red_greedy = []
      dot = Digraph(format='png')
      for head in greedy_db.names:
        dot.node(head)
      for edge in greedy_db.edges:
        parent = edge[0] 
        child = edge[1]
        dot.edge(parent, child)
      dot.render("red_greedy")
      file_image = open("red_greedy.png", "rb")
      image = file_image.read()
      image_widget = widgets.Image(value=image, format='png', layout=widgets.Layout(width='400px'))
      red_greedy.append(image_widget)
      params_greedy.pop()
      params_greedy.append(widgets.HBox(red_greedy))
      childs_greedy = []
      childs_greedy.append(widgets.VBox(params_greedy))
      left_greedy_vbox = widgets.HBox(childs_greedy)

      graph_greedy = []
      dgreedy = greedyLS(greedy_db, metrica_greedy_drop.value)
      dot = Digraph(format='png')
      for head in dgreedy.names:
        dot.node(head)
      for edge in dgreedy.edges:
        parent = edge[0] 
        child = edge[1]
        dot.edge(parent, child)
      dot.render("greedy")
      file_image = open("greedy.png", "rb")
      image = file_image.read()
      image_widget = widgets.Image(value=image, format='png', layout=widgets.Layout(width='400px'))
      graph_greedy.append(image_widget)
      right_greedy_vbox = widgets.VBox(graph_greedy)
      tab_greedy = widgets.HBox([left_greedy_vbox, right_greedy_vbox])
      list_tabs = list(tab_nest.children)
      list_tabs.pop(3)
      list_tabs.insert(3, tab_greedy)
      tab_nest.children = tuple(list_tabs)

    add_greedy_button.on_click(add_greedy_edge)
    delete_greedy_button.on_click(delete_greedy_edge)
    greedy_button.on_click(greedy_button_click)

    buttons_greedy = widgets.HBox([add_greedy_button, delete_greedy_button, greedy_button])
    params_greedy.append(buttons_greedy)

    params_greedy.append(widgets.HBox(red_greedy))

    childs_greedy.append(widgets.VBox(params_greedy))
    left_greedy_vbox = widgets.HBox(childs_greedy)
    right_greedy_vbox = widgets.VBox(graph_greedy)
    tab_greedy = widgets.HBox([left_greedy_vbox, right_greedy_vbox])

    # SHOW INFERENCIA
    dk2 = K2(db, 1, 1, 'K2')
    algBP = InterMensajes(dk2,[],[])
    algBP.InicializarArbol()

    childs_inter = list()
    params_inter = list()
    red_dataset_list = list()
    query_dataset_list = list()
    red_dataset_list= list()

    inter_button = widgets.Button(
      description='Calcular',
      disabled=False,
      button_style='info',
      tooltip='',
      icon='check')

    query_inter_button = widgets.Button(
      description='Consultar',
      disabled=False,
      button_style='info',
      tooltip='',
      icon='check')
    
    query_inter_param = widgets.Dropdown(options=db.names,
                              value=db.names[0],
                              description="Var Consulta:")
    
    def inter_button_click(b):
      red_dataset_list=[]
      for i in range(len(db.names)):
        head = db.names[i]
        red_labels=list()
        red_rows_boxes = list()

        red_label = widgets.Button(description = head, disabled = True, button_style='info')
        red_label_lambda = widgets.Button(description = "Lambda", disabled = True, button_style='info')
        red_label_pi = widgets.Button(description = "Pi", disabled = True, button_style='info')
        red_label_msg = widgets.Button(description = "L. Msg.", disabled = True, button_style='info')
        red_label_pi_msg = widgets.Button(description = "Pi Msg", disabled = True, button_style='info')
        red_labels.append(red_label)
        red_labels.append(red_label_lambda)
        red_labels.append(red_label_pi)
        red_labels.append(red_label_msg)
        red_labels.append(red_label_pi_msg)

        red_headers_box = widgets.HBox(red_labels)

        nvar, lambP, piP, lamb, pi, probs = algBP.query(db.mnames[head])
        for j in range(len(probs)):
          values = list()
          value = widgets.Button(description = db.uniq[i][j], disabled = True)
          lambda_widget = widgets.Button(description = (str(lambP[j]) if len(lambP) > 0 else ""), disabled = True)
          pi_widget = widgets.Button(description = (str(piP[j]) if len(piP) > 0 else ""), disabled = True)
          msg_widget = widgets.Button(description = (str(lamb[j]) if len(lamb) > 0 else ""), disabled = True)
          pi_msg_widget = widgets.Button(description = (str(pi[j]) if len(pi) > 0 else ""), disabled = True)
          values.append(value)
          values.append(lambda_widget)
          values.append(pi_widget)
          values.append(msg_widget)
          values.append(pi_msg_widget)
          red_rows_boxes.append(widgets.HBox(values))

        red_dataset_box = widgets.VBox([red_headers_box,*red_rows_boxes])
        red_dataset_list.append(red_dataset_box)
      right_inter_vbox = widgets.VBox(red_dataset_list)
      tab_inter = widgets.HBox([left_inter_vbox, right_inter_vbox])
      list_tabs = list(tab_nest.children)
      list_tabs.pop(4)
      list_tabs.insert(4, tab_inter)
      tab_nest.children = tuple(list_tabs)

    def query_inter_button_click(b):
      nvar, lambP, piP, lamb, pi, probs = algBP.query(db.mnames[query_inter_param.value])

      query_rows_boxes = list()
      query_labels=list()
      query_dataset_list=[]
      query_label = widgets.Button(description = query_inter_param.value, disabled = True, button_style='info')
      query_labels.append(query_label)
      query_label_prob = widgets.Button(description = "Prob", disabled = True, button_style='info')
      query_labels.append(query_label_prob)
      query_headers_box = widgets.HBox(query_labels)

      index = db.mnames[query_inter_param.value]
      for i in range(len(probs)):
        values=list()
        value = widgets.Button(description = db.uniq[index][i], disabled = True)
        values.append(value)
        prob = widgets.Button(description = str(probs[i]), disabled = True)
        values.append(prob)
        query_rows_boxes.append(widgets.HBox(values))
      query_dataset_box = widgets.VBox([query_headers_box,*query_rows_boxes])
      query_dataset_list.append(query_dataset_box)
      params_inter.pop()
      params_inter.append(widgets.HBox(query_dataset_list))
      childs_inter = []
      childs_inter.append(widgets.VBox(params_inter))
      left_inter_vbox = widgets.HBox(childs_inter)
      tab_inter = widgets.HBox([left_inter_vbox, right_inter_vbox])
      list_tabs = list(tab_nest.children)
      list_tabs.pop(4)
      list_tabs.insert(4, tab_inter)
      tab_nest.children = tuple(list_tabs)

    inter_button.on_click(inter_button_click)
    query_inter_button.on_click(query_inter_button_click)
    params_inter.append(inter_button)
    params_inter.append(query_inter_param)
    params_inter.append(query_inter_button)
    params_inter.append(widgets.HBox(query_dataset_list))
    params_inter.append(widgets.HBox(red_dataset_list))
    childs_inter.append(widgets.VBox(params_inter))
    left_inter_vbox = widgets.HBox(childs_inter)
    right_inter_vbox = widgets.VBox(red_dataset_list)
    tab_inter = widgets.HBox([left_inter_vbox, right_inter_vbox])

    tab_nest.children = [*tab_nest.children,dataset_box,tab_k2,tab_greedy,tab_inter]

btn_upl_data.on_click(upl_data_file)
btn_create.on_click(load_components)
hiper_param_slider.observe(on_change, names='value')

upl_data_box = widgets.HBox([upl_data,btn_upl_data])
upl_box = widgets.VBox([upl_data_box,btn_create])

tab_nest.children = [upl_box]
tab_nest.set_title(0, 'UPLOAD')
tab_nest.set_title(1, 'DATASET')
tab_nest.set_title(2, 'K2')
tab_nest.set_title(3, 'GREEDY')
tab_nest.set_title(4, 'INFERENCIA')
tab_nest

Archivo subido
